In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
pd.set_option("display.max_columns", 100)

In [24]:
analysis_date1 = '2022-05-25'
analysis_date2 = '2022-05-26'
itp_id = 300


In [47]:
updates = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
WHERE date BETWEEN '{analysis_date1}' AND '{analysis_date2}'
AND calitp_itp_id = {itp_id}
LIMIT 2000
''')

In [48]:
updates.head()

,calitp_itp_id,calitp_url_number,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key
0,300,0,gtfs-data/rt/2022-05-25T22:00:15/300/0/gtfs_rt...,2022-05-25,881417_1566_50700,1653515897,None,1566,None,None,881417,3481,0,14:05:00,20220525,SCHEDULED,"[{'stopSequence': 35, 'stopId': '4', 'arrival'...",e9b43b732e43cff6d5162fc2a5a867d3
1,300,0,gtfs-data/rt/2022-05-25T22:00:15/300/0/gtfs_rt...,2022-05-25,882105_1505_50520,1653515943,None,1505,None,None,882105,3486,0,14:02:00,20220525,SCHEDULED,"[{'stopSequence': 45, 'stopId': '405', 'arriva...",113c76a9e6a27948aeaf6f6b6717abb1
2,300,0,gtfs-data/rt/2022-05-25T22:00:15/300/0/gtfs_rt...,2022-05-25,883320_5314_50580,1653515975,None,5314,None,None,883320,3495,0,14:03:00,20220525,SCHEDULED,"[{'stopSequence': 16, 'stopId': '4', 'arrival'...",c1f91b5f1455c5c4adf2bdb593562b0b
3,300,0,gtfs-data/rt/2022-05-25T22:00:15/300/0/gtfs_rt...,2022-05-25,881533_2103_51000,1653515842,None,2103,None,None,881533,3483,0,14:10:00,20220525,SCHEDULED,"[{'stopSequence': 48, 'stopId': '1354', 'arriv...",32ae68a62c9cf7a338ccc49f70f6e92c
4,300,0,gtfs-data/rt/2022-05-25T22:00:15/300/0/gtfs_rt...,2022-05-25,882953_2905_51300,1653515947,None,2905,None,None,882953,3492,1,14:15:00,20220525,SCHEDULED,"[{'stopSequence': 31, 'stopId': '1470', 'arriv...",47a47b0d13461e1987ea388401b8b4a1


In [49]:
updates.trip_schedule_relationship.value_counts()

SCHEDULED    2000
Name: trip_schedule_relationship, dtype: int64

In [52]:
updates>>count(_.date)

,date,n
0,2022-05-25,2000


In [51]:
updates.trip_id.value_counts()

882285    12
881423    12
881898    12
880847    12
881060    12
          ..
881417     3
882521     3
881827     3
881896     3
882879     2
Name: trip_id, Length: 184, dtype: int64

In [53]:
updates.trip_direction_id.value_counts()

1    1004
0     996
Name: trip_direction_id, dtype: int64

In [54]:
updates.vehicle_id.value_counts()

1820                            35
1332                            34
2912                            34
2909                            33
1824                            33
                                ..
1317                            11
block_9503_schedBasedVehicle    11
2904                            11
1318                            11
1338                             3
Name: vehicle_id, Length: 101, dtype: int64

In [55]:
updates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   calitp_itp_id               2000 non-null   int64 
 1   calitp_url_number           2000 non-null   int64 
 2   original_file_path          2000 non-null   object
 3   date                        2000 non-null   object
 4   id                          2000 non-null   object
 5   timestamp                   2000 non-null   int64 
 6   delay                       0 non-null      object
 7   vehicle_id                  2000 non-null   object
 8   vehicle_label               0 non-null      object
 9   vehicle_license_plate       0 non-null      object
 10  trip_id                     2000 non-null   object
 11  trip_route_id               2000 non-null   object
 12  trip_direction_id           2000 non-null   int64 
 13  trip_start_time             2000 non-null   obje

In [56]:
# reading in the vehicle_position and scheduled trips
trips = utils.read_data()

In [57]:
# filtering down to same operator (BBB) and service date (2022-05-25)

trips = (trips >> filter(_.calitp_itp_id == itp_id,
                         _.service_date >= analysis_date1,
                         _.service_date <= analysis_date2))

In [58]:
trips>>count(_.service_date)

,service_date,n
0,2022-05-25,19
1,2022-05-26,19


In [59]:
len(trips)

38

In [60]:
trips.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
291,300,Big Blue Bus,0,3501,44,2022-05-25,104,0,0.000000,Wednesday,May
295,300,Big Blue Bus,0,3501,44,2022-05-26,104,0,0.000000,Thursday,May
6951,300,Big Blue Bus,0,3488,R10,2022-05-25,6,5,0.833333,Wednesday,May
7463,300,Big Blue Bus,0,3488,R10,2022-05-26,7,7,1.000000,Thursday,May
8674,300,Big Blue Bus,0,3483,5,2022-05-25,27,21,0.777778,Wednesday,May


In [90]:
updates.sample()

,calitp_itp_id,calitp_url_number,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key
623,300,0,gtfs-data/rt/2022-05-25T22:00:56/300/0/gtfs_rt...,2022-05-25,881546_2103_54900,1653515983,None,2103,None,None,881546,3483,1,15:15:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '1354', 'arriva...",eb99dc912208972d0caf392301d5cf84


In [99]:
## joining updates and trips

In [101]:
updates.date.value_counts()

2022-05-25    2000
Name: date, dtype: int64

In [102]:
updates>>count(_.date)

,date,n
0,2022-05-25,2000


In [105]:
updates.date.dtype

dtype('O')

In [110]:
import datetime

In [119]:
single_analysis_date = datetime.date(2022, 5, 25)

In [120]:
len(updates>>filter(_.date==single_analysis_date))

2000

In [121]:
join = (pd.merge((trips>>filter(_.service_date==analysis_date1)), 
                 (updates>>filter(_.date==single_analysis_date)),
                 how='outer', left_on='route_id', right_on='trip_route_id',
                 indicator='have_updates'))

#note changing from merging on trip_id to route_id

In [122]:
join.head()

,calitp_itp_id_x,agency_name,calitp_url_number_x,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month,calitp_itp_id_y,calitp_url_number_y,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key,have_updates
0,300,Big Blue Bus,0,3501,44,2022-05-25,104,0,0.000000,Wednesday,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,300,Big Blue Bus,0,3488,R10,2022-05-25,6,5,0.833333,Wednesday,May,300.0,0.0,gtfs-data/rt/2022-05-25T22:00:15/300/0/gtfs_rt...,2022-05-25,882324_3875_56100,1.653516e+09,None,3875,None,None,882324,3488,1.0,15:35:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '1317', 'arriva...",08ca584d9eb401d2dac5b14bce1c2c19,both
2,300,Big Blue Bus,0,3488,R10,2022-05-25,6,5,0.833333,Wednesday,May,300.0,0.0,gtfs-data/rt/2022-05-25T22:00:36/300/0/gtfs_rt...,2022-05-25,882324_3875_56100,1.653516e+09,None,3875,None,None,882324,3488,1.0,15:35:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '1317', 'arriva...",a8a4c0fafc0c2f30cbf55cdcc56d7a96,both
3,300,Big Blue Bus,0,3488,R10,2022-05-25,6,5,0.833333,Wednesday,May,300.0,0.0,gtfs-data/rt/2022-05-25T22:00:56/300/0/gtfs_rt...,2022-05-25,882324_3875_56100,1.653516e+09,None,3875,None,None,882324,3488,1.0,15:35:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '1317', 'arriva...",8c393d873930a1f9ba86060e2afef86d,both
4,300,Big Blue Bus,0,3488,R10,2022-05-25,6,5,0.833333,Wednesday,May,300.0,0.0,gtfs-data/rt/2022-05-25T22:01:16/300/0/gtfs_rt...,2022-05-25,882324_3875_56100,1.653516e+09,None,3875,None,None,882324,3488,1.0,15:35:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '1317', 'arriva...",a32723b72ad1c95ee63f146b6db5937a,both


In [123]:
join.have_updates.value_counts()

both          2000
left_only        1
right_only       0
Name: have_updates, dtype: int64

In [124]:
(join>>filter(_.have_updates=='left_only'))

,calitp_itp_id_x,agency_name,calitp_url_number_x,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month,calitp_itp_id_y,calitp_url_number_y,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key,have_updates
0,300,Big Blue Bus,0,3501,44,2022-05-25,104,0,0.0,Wednesday,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Note: 
* With a limit of 1000, there were three trips that did not match.
* With a limit of 2000 and two dates, there were two trips that did not match
* With a limit of 2000 and one date, there is one trip that did not match

In [130]:
join.sample()

,calitp_itp_id_x,agency_name,calitp_url_number_x,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month,calitp_itp_id_y,calitp_url_number_y,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key,have_updates
1767,300,Big Blue Bus,0,3479,1,2022-05-25,194,193,0.994845,Wednesday,May,300.0,0.0,gtfs-data/rt/2022-05-25T22:02:16/300/0/gtfs_rt...,2022-05-25,880752_1703_56400,1.653516e+09,None,1703,None,None,880752,3479,0.0,15:40:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '1305', 'arriva...",9beddba5cb6a6e7483581fb7109e56b5,both


In [127]:
(join>>filter(_.have_updates=='both')).delay.info()

<class 'pandas.core.series.Series'>
Int64Index: 2000 entries, 1 to 2000
Series name: delay
Non-Null Count  Dtype 
--------------  ----- 
0 non-null      object
dtypes: object(1)
memory usage: 31.2+ KB


In [131]:
join.delay.describe()

count       0
unique      0
top       NaN
freq      NaN
Name: delay, dtype: object

In [135]:
join.trip_schedule_relationship.value_counts()

SCHEDULED    2000
Name: trip_schedule_relationship, dtype: int64

In [126]:
join>>filter(_.have_updates=='both')>>count(_.trip_id)>>arrange(-_.n)

,trip_id,n
2,880752,12
6,880756,12
7,880757,12
9,880759,12
13,880837,12
...,...,...
35,881417,3
56,881827,3
61,881896,3
92,882521,3


In [70]:
join>>filter(_.have_updates=='both')>>count(_.route_id)>>arrange(-_.n)

,route_id,n
0,3479,534
4,3485,352
8,3489,342
14,3495,330
2,3481,304
9,3490,296
12,3493,248
1,3480,242
5,3486,220
6,3487,220


In [142]:
join.sample()

,calitp_itp_id_x,agency_name,calitp_url_number_x,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month,calitp_itp_id_y,calitp_url_number_y,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key,have_updates
69,300,Big Blue Bus,0,3491,15,2022-05-25,29,29,1.0,Wednesday,May,300.0,0.0,gtfs-data/rt/2022-05-25T22:02:16/300/0/gtfs_rt...,2022-05-25,882913_1820_53820,1.653516e+09,None,1820,None,None,882913,3491,1.0,14:57:00,20220525,SCHEDULED,"[{'stopSequence': 2, 'stopId': '140', 'arrival...",8184a5f8437edd229d7d8b3595dc875b,both


In [144]:
from shared_utils import geography_utils

In [166]:
updates_agg = (geography_utils.aggregate_by_geography(
    (join>>filter(_.have_updates=="both")),
    group_cols=["calitp_itp_id_x",
                 "agency_name",
                 "calitp_url_number_x",
                "route_id",
                "route_short_name",
                 "service_date",
                "num_sched",
                "num_vp",
                "pct_w_vp",
                 "weekday",
                 "month"],
    count_cols= ["trip_route_id"],
    nunique_cols = ["delay"],
    rename_cols = True,
)).rename(columns = {'trip_route_id_count':'num_trip_updates'})

In [179]:
updates_agg_subset = updates_agg>>select(_.agency_name, _.service_date, _.num_sched, _.num_vp, _.num_trip_updates)

In [177]:
import altair as alt

In [186]:
updates_agg_long =  (updates_agg_subset>>select(_.agency_name,
                                     _.service_date,
                                     _.num_sched,
                                     _.num_vp,
                                     _.num_trip_updates
                        ) 
             >> gather('measure',
                       'value',
                       _.num_sched,
                       _.num_vp,
                       _.num_trip_updates
                      )
            )

In [188]:
updates_agg_long.sample()

,agency_name,service_date,measure,value
7,Big Blue Bus,2022-05-25,num_sched,60


In [195]:
alt.Chart(updates_agg_long).mark_bar().encode(
    x='measure',
    y='value',
    color='measure',
)

alt.Chart(...)